In [1]:
import os
import cv2
import math
import time
import datetime
import warnings
import numpy as np
import matplotlib.pyplot as plt
import PySimpleGUI as sg
import pandas as pd

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib
matplotlib.use('Agg')

- Previous Function

In [2]:
def detectColor(img, lower, upper): 
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv.copy(), lower, upper)
        res = cv2.bitwise_and(img, img, mask= mask)
        return res

In [3]:
def findContourCircle(res): 
        x, y, r = 0, 0, 0
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contours = [cnt for cnt in contours if contour_ok(cnt)]
        circle = []
        for cnt in contours:
            (x,y), radius = cv2.minEnclosingCircle(cnt)
            if radius > 10: 
                circle.append([int(x), int(y), int(radius)])
        if len(circle) > 0 :
            circle = np.array(circle)
            x = int(np.mean(circle[:, 0]))
            y = int(np.mean(circle[:, 1]))
            r = int(np.mean(circle[:, 2]))
        return x, y, r
    
def contour_ok(cnt):
    x,y,w,h = cv2.boundingRect(cnt)
    
    return not (w < 30 or h < 30)    

In [4]:
def calc_speed(x, y, t):
    dx = float(x[1]) - float(x[0])
    dy = float(y[1]) - float(y[0])
    ds = math.sqrt( math.pow(dx,2) + math.pow(dy,2))
    dt = float(t[1]) - float(t[0])
    v = float(ds/dt)
    vx = float(dx/dt)
    vy = float(dy/dt)
    return v, vx, vy

In [5]:
color_ranges = dict(
    blue = dict(
        lower = np.array([110, 50, 50]),
        upper = np.array([130, 255, 255])
    ),
    orange = dict(
        lower = np.array([10, 50, 50]),
        upper = np.array([20, 255, 255])
    ) 
)

availabel_colors = color_ranges.keys()

In [6]:
def readBytesFromFrame(cap):
    frame_byte = None
    ret, frame = cap.read()

    if ret :
        frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        ret, frame_png = cv2.imencode('.png', frame)
        frame_byte = frame_png.tobytes()
    return ret, frame_byte

In [7]:
def create_custom_button(image_filename, key, disabled=False):
    return sg.Button(key=key, 
                      disabled=disabled, 
                      image_filename=image_filename, 
                      button_color=('#FFF', '#FFF'), 
                      image_size=(40, 40), 
                      image_subsample=3, 
                      border_width=0)

In [8]:
def clean_video(cap):
    if cap is not None:
        cap.release()
        cap = None
    window['-IMAGE-'].update(filename='ready_video.png')
    return cap

In [9]:
import json

def readJson_config(Path, Name, Key):
    with open(Path + Name) as json_config:
        json_object = json.load(json_config)

    return json_object[Key]


def writeJson_config(Path, Name, Data, append):
    mode = 'a+' if append else 'w'
    full_path = Path + Name

    with open(full_path, mode=mode) as json_config:
        json.dump(Data, json.load(json_config) if append else json_config)
    
    return 'success' 

def updateJson_config(Path, Name, Key, Value):
    with open(Path + Name) as json_config:
        json_object = json.load(json_config)

    json_object[Key] = Value
    
    with open(Path + Name, mode='w') as json_config:
        json.dump(json_object, json_config)
        
    return 'success' 

In [10]:
def check_setting():
    return os.path.exists("./setting.json")

In [11]:
def default_setting():
    if not check_setting():
        Data = dict(
            FPS = int((1/10)*1000),  # for 25ms delay
            Format_File = "*.MP4",
            x0 = 0,
            y0 = 0,
            Scale = 10,
            Ratio = float('%.2f' % (30/700)) # 30 cm as 700 pixel
        )

        writeJson_config("./", "setting.json", Data, False)
        
default_setting()

In [12]:
def create_input_setting(width=25, key='', setting_key=''):
    return sg.In(size=(width, 1), 
            key=key, 
            default_text=readJson_config("./", "setting.json", setting_key))

In [13]:
def readBytesFromDetectedObjectFrame(cap):
    global i, t0, x0, y0 
    frame_byte = None
    v_, vx_, vy_, x_, y_ = 0, 0, 0, 0, 0
    center = [0, 0]
    ret, img = cap.read()
    if ret:
        h, w, c = img.shape
        color = color_ranges[color_input]
        res = detectColor(img, color['lower'], color['upper'])
        
        x, y, r = findContourCircle(res)
        try :
            cv2.circle(img, (x, y), r, (255, 0, 255), 2)

            center = [x, y]
            cv2.putText(img, "(%d, %d)" % (x, y), (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA) 
            xs.append(x) 
            ys.append(y) 

            if i % 15: 
                t = time.time()
                x_ = np.mean(xs[-60:]) 
                y_ = np.mean(h - np.array(ys[-60:])) 
                v, vx, vy = calc_speed([x0, x_], [y0, y_], [t0, t])
                v_, vx_, vy_ = v*ratio, vx*ratio, vy*ratio

                t0, y0, x0 = t, y_, x_

                #x, y = x - r, y + r #center marker adjusment
                cv2.line(img, (x,y), (x, y - r), (200, 200, 200), 2)
                cv2.line(img, (x,y), (x + r, y), (200, 200, 200), 2)
                cv2.arrowedLine(img, (x, y), (x, y - int(vy*scale)), (0,255,255), 2, cv2.LINE_AA) 
                cv2.arrowedLine(img, (x, y), (x - int(vx*scale), y), (0,255,0), 2, cv2.LINE_AA) 
                cv2.arrowedLine(img, (x, y), (x - int(vx*scale), y - int(vy*scale)), (0,0,255), 2, cv2.LINE_AA) 
                if v > 2 :
                    cv2.putText(img, "%.2f cm/s" % v_, (x - int(vx*scale), y - int(vy*scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA) 
                trajectory.append([x,y]) 
            
            # draw trajectory              
            if len(trajectory) > 60 and is_view_trajectory: 
                vec_trajectory = np.array(trajectory)
                cv2.polylines(img, [vec_trajectory], False, (255,255,0), 1, cv2.LINE_AA)
                
                xc = vec_trajectory[:,0]  
                yc = vec_trajectory[:,1]  
                
                # draw trajectory interplation                
                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try :
                        orde = 2 if len(xc) <= 100 else 3 if len(xc) > 100 and len(xc) < 500 else 5 
                        poly_solve = np.unique(yc), np.int0(np.poly1d(np.polyfit(yc, xc, orde))(np.unique(yc)))  
                        cv2.polylines(img, [np.array(list(zip(poly_solve[1], poly_solve[0])))], False, (0,0,255), 1, cv2.LINE_AA)  
                    except np.RankWarning:
                        pass
            
            # scale bar
            (x1, y1), (x2, y2) = (int(w*0.1), int(h*0.8)), (int(w*0.1), int(h*0.9))
            (xm, ym) = (int(w*0.13), int(h*0.85 + 8))
            cv2.line(img, (x1, y1), (x2, y2), (200), 2)
            cv2.putText(img, "%.2f cm" % ((y2-y1)*ratio) , (xm, ym), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
        
        except Exception as e: 
            print("[ERROR] Error : %s" % e)
            pass
        
        #img = cv2.resize(img, (0,0), fx=0.7, fy=0.7)
        img = img[180:, :]#img[300:, 100:-100]  # [y0:y1, x0:x1]
        
        ret, frame_png = cv2.imencode('.png', img)
        frame_byte = frame_png.tobytes()

        del xs[:-120]
        del ys[:-120]
        i += 1 

    return ret, frame_byte, (v_, vx_, vy_), center, (x_, y_)

In [14]:
def draw_figure_plot(canvas, figure):
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side="top", fill="both", expand=1)
    return figure_canvas_agg

In [17]:
image_play = "asset/play.png"
image_pause = "asset/pause.png"
image_stop = "asset/stop.png"

menu_def = [['Home', 'Home'],      
            ['Setting', 'Setting'],      
            ['Help', 'About...']]

file_list_column = [
    [
        sg.Text("Video Folder"),
        sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
        sg.FolderBrowse(),
    ],
    [
        sg.Listbox(values=[], enable_events=True, size=(40, 28), key="-FILE LIST-")
    ],
]

# ------------------- tab layout -------------------------
image_viewer_tab = [
    [
    create_custom_button(image_play, key='play', disabled=True),
    create_custom_button(image_pause, key='pause', disabled=True),
    create_custom_button(image_stop, key='stop', disabled=True),
    sg.Checkbox('View Trajectory', key='-view_trajectory-', enable_events=True),
    sg.Combo(['blue', 'orange'], default_value='blue', size = (10, 1), key='-color-', enable_events=True)
    ],
    [sg.Image(filename='default_video.png', key="-IMAGE-")]
]

chart_viewer_tab = [
    [sg.Canvas(key="-plot-data-", size=(500, 350))]
]

tab_group_column = [[sg.TabGroup([[sg.Tab('Video', image_viewer_tab), 
                                  sg.Tab('Chart', chart_viewer_tab)]])],
                    [sg.Column([
                        [sg.Text("Position \t: x(0), y(0)", key='-position_label-', size=(50, None))],
                        [sg.Text("time \t: 00:00:00 ", key='-time_label-', size=(25, None))],
                        [sg.Text("v \t: 0 cm/s", key='-v_label-', size=(25, None))],
                        [sg.Text("vx \t: 0 cm/s", key='-vx_label-', size=(25, None))],
                        [sg.Text("vy \t: 0 cm/s", key='-vy_label-', size=(25, None))]
                    ], size=(350, 130)),
                     sg.Column([
                         [sg.Button('Save Result')]
                     ])
                    ],
                   ]
# ------------------- end tab layout -------------------------


layout_home = [
    [
        sg.Column(file_list_column),
        sg.VSeperator(),
        sg.Column(tab_group_column),
    ]
]

layout_setting = [
    [sg.Text('FPS \t\t: ', size=(15, None)), create_input_setting(width=25, key='-fps-', setting_key='FPS')],
    [sg.Text('Format File \t: ', size=(15, None)), create_input_setting(width=25, key='-format_file-', setting_key='Format_File')],
    [sg.Text('Position \t\t: ', size=(15, None)), 
         sg.Text('x0 : ', size=(3, None)), create_input_setting(width=5, key='-x0-', setting_key='x0'),
         sg.Text('y0 : ', size=(3, None)), create_input_setting(width=5, key='-y0-', setting_key='y0')],
    [sg.Text('Scale \t\t: ', size=(15, None)), create_input_setting(width=25, key='-scale-', setting_key='Scale')],
    [sg.Text('Ratio (cm/s) \t: ', size=(15, None)), create_input_setting(width=25, key='-ratio-', setting_key='Ratio')],
    [sg.Button('Save')]
]

layout = [[sg.Menu(menu_def, )], 
          [sg.Column(layout_home, key='-home-'), sg.Column(layout_setting, visible=False, key='-setting-')]]

In [18]:
window = sg.Window("Video Viewer + Play List", finalize=True, layout=layout, margins=(10, 10))

window.set_min_size((700,400))

cap = None
video_path = ''
filename = ''
stat = ''

default_setting()
t0 = time.time()
x0 = int(readJson_config("./", "setting.json", "x0")) 
y0 = int(readJson_config("./", "setting.json", "y0"))
y0_cm = 0
scale = float(readJson_config("./", "setting.json", "Scale"))
ratio = float(readJson_config("./", "setting.json", "Ratio"))
fps = int(readJson_config("./", "setting.json", "FPS"))

color_input = 'blue'
is_view_trajectory = False

trajectory = [] 
xs, ys = [], [] 
i = 0
frame_n = 0

history = []

# -------- initialize figure as agg canvas -------
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlabel("Waktu")
ax.set_ylabel("Jarak (cm)")
ax.set_title("Plot Distance (cm) vs Time (s)")
ax.grid(True)
figure_canvas_agg = draw_figure_plot(window["-plot-data-"].TKCanvas, fig)

while True:
    event, values = window.read(timeout=int(1000/fps))
    if event == sg.WIN_CLOSED:
        break
        
    if event == "Home" :
        window['-home-'].update(visible=True)
        window['-setting-'].update(visible=False)
        
    if event == "Setting" :
        window['-home-'].update(visible=False)
        window['-setting-'].update(visible=True)
        
    if event == "About...":
        sg.popup('About', 'Version 1.0', 'OpenCV Object Tracking')
        
    if event == "Save" :
        updateJson_config("./", "setting.json", "FPS", values["-fps-"])
        updateJson_config("./", "setting.json", "Format_File", values["-format_file-"])
        updateJson_config("./", "setting.json", "x0", values["-x0-"])
        updateJson_config("./", "setting.json", "y0", values["-y0-"])
        updateJson_config("./", "setting.json", "Ratio", values["-ratio-"])
        updateJson_config("./", "setting.json", "Scale", values["-scale-"])
        
        global x0, y0, scale, ratio
        x0, y0 = int(readJson_config("./", "setting.json", "x0")), int(readJson_config("./", "setting.json", "y0"))
        scale = float(readJson_config("./", "setting.json", "Scale"))
        ratio = float(readJson_config("./", "setting.json", "Ratio"))
        fps = int(readJson_config("./", "setting.json", "FPS"))
          
        sg.popup('Setting saved successfully!')
        
    if event == "Save Result" :
        df = pd.DataFrame(history, columns=['Time', "Frame", "V (cm/s)", "Vx (cm/s)", "Vy (cm/s)", "X (cm)", "Y (cm)"])
        df.to_csv('Result-%s.csv' % filename, index=None)
        sg.popup('Result saved successfully!')
        
    if event == "-view_trajectory-" :
        is_view_trajectory = values['-view_trajectory-']
        
    if event == "-color-" :
        color_input = values['-color-']
        cap = clean_video(cap)
        stat = 'stop'
        #trajectory = []
        history = []
        window['play'].update(disabled=False)
        window['pause'].update(disabled=True)
        window['stop'].update(disabled=True) 
    
    if event == "-FOLDER-":
        folder = values["-FOLDER-"]
        try:
            file_list = os.listdir(folder)
        except:
            file_list = []
            
        fnames = [
            f
            for f in file_list
            if os.path.isfile(os.path.join(folder, f))
            and f.lower().endswith(("video", ".mp4"))
        ]
        
        window["-FILE LIST-"].update(fnames)
        window['-IMAGE-'].update(filename='ready_video.png')
    
    elif event == "-FILE LIST-":  
        try:
            folder = values["-FOLDER-"]
            filename = values["-FILE LIST-"][0]
            video_path = os.path.join(folder, filename)
            
            cap = clean_video(cap)
            window['play'].update(disabled=False)
            window['pause'].update(disabled=True)
            window['stop'].update(disabled=True) 
        except:
            pass
        
    elif event == 'play' or event == 'pause' or event == 'stop':
        stat = event
        if event == 'play' :
            trajectory = []
            history = []
            window['play'].update(disabled=True)
            window['pause'].update(disabled=False)
            window['stop'].update(disabled=False)
        elif event == 'pause' or event == 'stop':
            window['play'].update(disabled=False)
            window['pause'].update(disabled=True)
            window['stop'].update(disabled=True) 
        
    if stat == 'play':
        curr_time = datetime.datetime.now().strftime("%H:%M:%S.%f")

        if cap is None:
            cap = cv2.VideoCapture(video_path)
            
        ret, frame_byte, (v_, vx_, vy_), (x, y), (x_, y_) = readBytesFromDetectedObjectFrame(cap)
        
        if ret:
            frame_n = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        else :
            frame_n = 0
            
        if x_ != 0 or y_ != 0 :
            if len(history) < 1 :
                y0_cm = y_*ratio
            history.append([curr_time, frame_n, v_, vx_, vy_, x_*ratio, y_*ratio]) # all variable in cm 
        
        if not ret:
            cap = clean_video(cap)
            stat = 'stop'
            trajectory = []
            history = []
            window['play'].update(disabled=False)
            window['pause'].update(disabled=True)
            window['stop'].update(disabled=True) 
          
        # -- draw video frame ---- 

        window['-IMAGE-'].update(data=frame_byte)
        window['-position_label-'].update(value='Position \t: x(%d cm), y(%d cm), frame (%d)' % (x_*ratio, y_*ratio, frame_n))
        window['-time_label-'].update(value='time \t: %s' % curr_time)
        window['-v_label-'].update(value='v \t: %.2f cm/s' % v_)
        window['-vx_label-'].update(value='vx \t: %.2f cm/s' % vx_)
        window['-vy_label-'].update(value='vy \t: %.2f cm/s' % vy_)
        
        # -- draw chart ---- 
        # draw each 15 frame
        if ret :
            curr_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
            if curr_frame%15 == 0 :
                idx = np.arange(len(history))
                sampling = int(len(idx)/10)
                if sampling > 1 :
                    ax.cla()
                    ax.grid(True)
                    ax.set_xlabel("x (cm)")
                    ax.set_ylabel("y (cm)")
                    ax.set_title("Scatter Plot X - Y")
                    data = pd.DataFrame(history).values #[idx%sampling == 0]
                    #tm = data[:, 0]
                    #tm = [datetime.datetime.strptime(t, '%H:%M:%S.%f').strftime("%H:%M:%S") for t in tm]
                    
                    ax.set_ylim(max(0, min(data[:, -1]) -2), max(data[:, -1]) + 2)
                    ax.set_xlim(max(0, min(data[:, -2]) -2), max(data[:, -2]) + 2)
                    
                    yy = data[:, -1] #- y0_cm
                    xx = data[:, -2]
                    ax.scatter(xx, yy, marker='.')
                    figure_canvas_agg.draw()
    elif cap is not None and stat == 'stop':
        cap = clean_video(cap)

        
window.close()